In [38]:
import numpy as np
import pandas as pd
import networkx as nx

In [3]:
pd.read_csv("../data/cora/cites.tsv", sep="\t")

,cited_paper_id,citing_paper_id
0,35,887
1,35,1033
2,35,1688
3,35,1956
4,35,8865
5,35,12576
6,35,15670
7,35,18582
8,35,27510
9,35,28290


In [41]:
with open("../data/cora/cites.tsv", "rb") as inf:
    next(inf, "")
    G = nx.read_edgelist(inf, delimiter="\t", )

In [42]:
len(G)

2708

In [43]:
nx.number_connected_components(G)

78

In [49]:
node_map = {n : i for i, n in enumerate(G.nodes())}

In [50]:
node_map

{u'74700': 0,
 u'684986': 1,
 u'262108': 454,
 u'23448': 4,
 u'131117': 5,
 u'144330': 6,
 u'193354': 7,
 u'1109199': 246,
 u'193352': 8,
 u'1121867': 9,
 u'1105505': 10,
 u'103543': 11,
 u'1104055': 12,
 u'469504': 13,
 u'6151': 16,
 u'6152': 17,
 u'189572': 18,
 u'6155': 19,
 u'43186': 167,
 u'287787': 21,
 u'289780': 22,
 u'277263': 187,
 u'1106287': 24,
 u'158098': 25,
 u'69392': 26,
 u'1132434': 27,
 u'69397': 28,
 u'65650': 29,
 u'65653': 30,
 u'8696': 2082,
 u'3237': 1817,
 u'100701': 31,
 u'1102442': 32,
 u'689439': 33,
 u'853114': 34,
 u'853115': 35,
 u'853116': 36,
 u'52003': 37,
 u'52000': 38,
 u'853118': 39,
 u'52007': 40,
 u'1129015': 41,
 u'1129018': 42,
 u'1132505': 43,
 u'43698': 44,
 u'436796': 1779,
 u'85324': 45,
 u'1113438': 46,
 u'166825': 2593,
 u'28851': 1363,
 u'63931': 49,
 u'93320': 50,
 u'990075': 51,
 u'2440': 52,
 u'644494': 53,
 u'1130568': 54,
 u'1109392': 55,
 u'1130567': 56,
 u'1106630': 57,
 u'1117184': 58,
 u'93923': 59,
 u'294145': 60,
 u'102939': 61

In [71]:
cora_labels = pd.read_csv("../data/cora/paper.tsv", sep="\t", )
cora_labels

,paper_id,class_label
0,35,Genetic_Algorithms
1,40,Genetic_Algorithms
2,114,Reinforcement_Learning
3,117,Reinforcement_Learning
4,128,Reinforcement_Learning
5,130,Reinforcement_Learning
6,164,Theory
7,288,Reinforcement_Learning
8,424,Rule_Learning
9,434,Reinforcement_Learning


In [70]:
label_map = {"Genetic_Algorithms" : 0, "Reinforcement_Learning" : 1, "Theory" : 2, 
            "Rule_Learning" : 3, "Case_Based" : 4, "Probabilistic_Methods" : 5, "Neural_Networks" : 6}

In [72]:
cora_label_matrix = lil_matrix((2708, 7))
for r in cora_labels.iterrows():
    paper_id = str(r[1]["paper_id"])
    paper_id = node_map[paper_id]
    class_label = r[1]["class_label"]
    class_label = label_map[class_label]
    cora_label_matrix[paper_id, class_label] = 1

In [73]:
cora_label_matrix = csr_matrix(cora_label_matrix)

In [74]:
sp.sparse.save_npz("../data/cora/paper_labels.npz", cora_label_matrix)

In [8]:
cora_content = pd.read_csv("../data/cora/content.tsv", sep="\t", )
cora_content

,paper_id,word_cited_id
0,35,word100
1,35,word1152
2,35,word1175
3,35,word1228
4,35,word1248
5,35,word1250
6,35,word1306
7,35,word141
8,35,word192
9,35,word20


In [10]:
import scipy as sp

In [57]:
from scipy.sparse import lil_matrix, csr_matrix

In [78]:
cora_content_matrix = lil_matrix((2708, 1433))

In [52]:
for r in cora_content.iterrows():
    paper_id = str(r[1]["paper_id"])
    paper_id = node_map[paper_id]
    word_cited_id = r[1]["word_cited_id"]
    word_cited_id = int(word_cited_id.split("word")[1]) - 1
    cora_content_matrix[paper_id, word_cited_id] = 1

In [58]:
cora_content_matrix = csr_matrix(cora_content_matrix)

In [65]:
sp.sparse.save_npz("../data/cora/cited_words.npz", cora_content_matrix)

In [68]:
sp.sparse.load_npz("../data/cora/cited_words.npz")

<2708x1433 sparse matrix of type '<type 'numpy.float64'>'
	with 53218 stored elements in Compressed Sparse Row format>